In [14]:
import pandas as pd
import pickle
from pymongo import MongoClient
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin_min
import numpy as np
from datetime import datetime

In [37]:
collection = 'noaa_nam_2'

db = MongoClient("lattice-100", 27018)
no_of_records = 7431587

pipeline = [
    {
        "$sample": {"size": no_of_records // 100}
    }
]

# work1.aggregate(pipe, allowDiskUse=True)

cursor = db.sustaindb[collection].aggregate(pipeline, allowDiskUse=True)
df_original = pd.DataFrame(list(cursor))
df_original.head()

,_id,year_month_day_hour,timestep,gis_join,latitude,longitude,mean_sea_level_pressure_pascal,surface_pressure_surface_level_pascal,orography_surface_level_meters,temp_surface_level_kelvin,...,10_metre_v_wind_component_meters_per_second,total_precipitation_kg_per_squared_meter,water_convection_precipitation_kg_per_squared_meter,soil_temperature_kelvin,pressure_pascal,visibility_meters,precipitable_water_kg_per_squared_meter,total_cloud_cover_percent,snow_depth_meters,ice_cover_binary
0,60909a0d5f950e5ec41fd2df,2010011106,6,G3901630,39.278990,-82.564242,102610.0,99273.0,257.725159,265.507339,...,4.118362,0.0,0.0,268.669220,23961.216736,24233.964914,6.767937,100.0,0.13808,0.0
1,60909cbc110610849de4f14a,2010011718,0,G2800950,33.692856,-88.542065,100793.0,99967.0,67.725159,284.547745,...,-1.654191,0.0,0.0,281.750107,11560.531616,24232.722563,21.787668,100.0,0.00000,0.0
2,60909994425466e808649940,2010011006,3,G3500350,32.682973,-106.047402,102947.0,88652.0,1222.975159,271.068878,...,0.848175,0.0,0.0,275.814087,18360.504150,24036.230493,4.066518,8.0,0.00000,0.0
3,60909bc2c2024e104b034e1c,2010011506,6,G5600450,43.954653,-104.984500,102621.0,85922.0,1425.725159,266.673477,...,2.516348,0.0,0.0,269.638870,24067.593384,24036.708409,4.661869,0.0,0.03568,0.0
4,60909c024f82b233cc060cb0,2010011600,0,G4100350,42.195058,-120.901195,101653.0,83390.0,1638.225159,274.019928,...,5.397832,0.0,0.0,273.021439,20267.579651,24234.115648,7.411194,100.0,0.03544,0.0


In [38]:
def format_date(t):
    t = str(t)
    date = datetime.strptime(t, '%Y%m%d%H')
    return f'{date.year}-{date.month}-{date.day} {str(date.hour).zfill(2)}:{str(date.minute).zfill(2)}:{str(date.second).zfill(2)}'


df = df_original
df['year_month_day_hour'] = df['year_month_day_hour'].apply(format_date)
print(df.shape)
df.head()

(74315, 24)


,_id,year_month_day_hour,timestep,gis_join,latitude,longitude,mean_sea_level_pressure_pascal,surface_pressure_surface_level_pascal,orography_surface_level_meters,temp_surface_level_kelvin,...,10_metre_v_wind_component_meters_per_second,total_precipitation_kg_per_squared_meter,water_convection_precipitation_kg_per_squared_meter,soil_temperature_kelvin,pressure_pascal,visibility_meters,precipitable_water_kg_per_squared_meter,total_cloud_cover_percent,snow_depth_meters,ice_cover_binary
0,60909a0d5f950e5ec41fd2df,2010-1-11 06:00:00,6,G3901630,39.278990,-82.564242,102610.0,99273.0,257.725159,265.507339,...,4.118362,0.0,0.0,268.669220,23961.216736,24233.964914,6.767937,100.0,0.13808,0.0
1,60909cbc110610849de4f14a,2010-1-17 18:00:00,0,G2800950,33.692856,-88.542065,100793.0,99967.0,67.725159,284.547745,...,-1.654191,0.0,0.0,281.750107,11560.531616,24232.722563,21.787668,100.0,0.00000,0.0
2,60909994425466e808649940,2010-1-10 06:00:00,3,G3500350,32.682973,-106.047402,102947.0,88652.0,1222.975159,271.068878,...,0.848175,0.0,0.0,275.814087,18360.504150,24036.230493,4.066518,8.0,0.00000,0.0
3,60909bc2c2024e104b034e1c,2010-1-15 06:00:00,6,G5600450,43.954653,-104.984500,102621.0,85922.0,1425.725159,266.673477,...,2.516348,0.0,0.0,269.638870,24067.593384,24036.708409,4.661869,0.0,0.03568,0.0
4,60909c024f82b233cc060cb0,2010-1-16 00:00:00,0,G4100350,42.195058,-120.901195,101653.0,83390.0,1638.225159,274.019928,...,5.397832,0.0,0.0,273.021439,20267.579651,24234.115648,7.411194,100.0,0.03544,0.0


In [39]:
df1 = df[["gis_join", "mean_sea_level_pressure_pascal"]]
df1

,gis_join,mean_sea_level_pressure_pascal
0,G3901630,102610.0
1,G2800950,100793.0
2,G3500350,102947.0
3,G5600450,102621.0
4,G4100350,101653.0
...,...,...
74310,G3001050,102381.0
74311,G3000150,101310.0
74312,G2601250,102389.0
74313,G0600050,100563.0


In [41]:
len(df1['gis_join'].unique())

3050

## Aggregate on 'mean_sea_level_pressure_pascal' for each GISJoin

In [42]:
df1 = pd.DataFrame(df.groupby("gis_join")["mean_sea_level_pressure_pascal"].mean())
dfX = df1['mean_sea_level_pressure_pascal']
df1

,mean_sea_level_pressure_pascal
gis_join,
G0100010,102425.647059
G0100030,102093.947368
G0100050,102158.055556
G0100070,102367.266667
G0100090,102311.764706
...,...
G5600370,102812.565385
G5600390,102472.652174
G5600410,102382.142857


In [43]:
dfX = np.array(dfX).reshape(-1, 1)

In [44]:
kmeans = KMeans(n_clusters=56, random_state=0).fit(dfX)
df_predict = kmeans.fit_predict(dfX)
centroids = kmeans.cluster_centers_
df1['cluster_id'] = df_predict
df1

,mean_sea_level_pressure_pascal,cluster_id
gis_join,,
G0100010,102425.647059,1
G0100030,102093.947368,6
G0100050,102158.055556,30
G0100070,102367.266667,24
G0100090,102311.764706,3
...,...,...
G5600370,102812.565385,17
G5600390,102472.652174,54
G5600410,102382.142857,49


In [45]:
len(centroids)

56

In [46]:
cluster_ids_list = df1['cluster_id']
centroids_column = []

for id in cluster_ids_list:
    centroids_column.append(centroids[id][0])
    
print(len(centroids_column))

3050


In [47]:
df1['centroid'] = centroids_column

In [49]:
df1['distance'] = abs(df1['mean_sea_level_pressure_pascal'] - df1['centroid'])
df1

,mean_sea_level_pressure_pascal,cluster_id,centroid,distance
gis_join,,,,
G0100010,102425.647059,1,102424.036754,1.610305
G0100030,102093.947368,6,102095.879548,1.932180
G0100050,102158.055556,30,102149.845348,8.210208
G0100070,102367.266667,24,102364.376036,2.890630
G0100090,102311.764706,3,102311.119751,0.644955
...,...,...,...,...
G5600370,102812.565385,17,102832.963118,20.397733
G5600390,102472.652174,54,102457.788657,14.863517
G5600410,102382.142857,49,102392.008194,9.865337


In [50]:
df2 = df1[["cluster_id", "distance"]].reset_index()
df2

,gis_join,cluster_id,distance
0,G0100010,1,1.610305
1,G0100030,6,1.932180
2,G0100050,30,8.210208
3,G0100070,24,2.890630
4,G0100090,3,0.644955
...,...,...,...
3045,G5600370,17,20.397733
3046,G5600390,54,14.863517
3047,G5600410,49,9.865337
3048,G5600430,34,8.018696


### Tag Parent GISJoins

In [51]:
df2_gr = df2.groupby('cluster_id')['distance'].min().reset_index()
parents = []
for i, row in df2_gr.iterrows():
    cluster_id = row['cluster_id']
    distance = row['distance']
    
    cluster_id_match = df2[df2['cluster_id'] == cluster_id]
    distance_match = cluster_id_match[cluster_id_match['distance'] == distance].reset_index()['gis_join']
    parent = list(distance_match)[0] # select only one GISJOIN
    parents.append(parent)
    
print(len(set(parents)))

56


In [52]:
df_clusters = df2_gr.rename(columns={'distance': 'min_distance'})
df_clusters['max_distance'] = df2.groupby('cluster_id')['distance'].max().reset_index()['distance']
print(df_clusters.shape)
df_clusters.head()

(56, 3)


,cluster_id,min_distance,max_distance
0,0,0.074699,19.675301
1,1,0.391817,16.263246
2,2,3.165837,75.768401
3,3,0.056720,13.665206
4,4,12.211255,12.211255


In [53]:
for parent in parents:
    df2.loc[df2.gis_join == parent, "is_parent"] = 1

df2 = df2.fillna(0)
df2.head()

,gis_join,cluster_id,distance,is_parent
0,G0100010,1,1.610305,0.0
1,G0100030,6,1.932180,0.0
2,G0100050,30,8.210208,0.0
3,G0100070,24,2.890630,0.0
4,G0100090,3,0.644955,0.0


In [56]:
df3 = df2

for i, row in df2.iterrows():
    i_distance = row['distance']
    cluster_id = row['cluster_id']
    gis_join = row['gis_join']
    max_distance = df_clusters[df_clusters['cluster_id'] == cluster_id]['max_distance'].item()
    min_distance = df_clusters[df_clusters['cluster_id'] == cluster_id]['min_distance'].item()
    if max_distance == min_distance:
        frac = 0
    else:
        frac = (i_distance - min_distance)/(max_distance - min_distance)
    df3.loc[df3.gis_join == gis_join, "frac_distance"] = frac

In [57]:
sample_min = 0.05
sample_max = 0.25

df3['sample_percent'] = sample_min + (sample_max - sample_min) * df3['frac_distance']
df3

,gis_join,cluster_id,distance,is_parent,frac_distance,sample_percent
0,G0100010,1,1.610305,0.0,0.076772,0.065354
1,G0100030,6,1.932180,0.0,0.148643,0.079729
2,G0100050,30,8.210208,0.0,0.618242,0.173648
3,G0100070,24,2.890630,0.0,0.213755,0.092751
4,G0100090,3,0.644955,0.0,0.043226,0.058645
...,...,...,...,...,...,...
3045,G5600370,17,20.397733,0.0,0.719611,0.193922
3046,G5600390,54,14.863517,0.0,0.882149,0.226430
3047,G5600410,49,9.865337,0.0,0.624436,0.174887
3048,G5600430,34,8.018696,0.0,0.453498,0.140700


In [58]:
unique_parents1 = list(df3[df3['is_parent'] == 1]['gis_join'].unique())
print(len(unique_parents1))

56


In [59]:
df_cluster_parents = df3[df3['is_parent'] == 1][['cluster_id', 'gis_join']].sort_values('cluster_id').reset_index()
del df_cluster_parents['index']
cluster_parent_map = {}

for i, row in df_cluster_parents.iterrows():
    cluster_parent_map[row['cluster_id']] = row['gis_join']

In [60]:
cluster_parent_map.keys()

dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55])

In [61]:
cluster_parent_map.values()

dict_values(['G5500290', 'G1800350', 'G5300570', 'G3101070', 'G0201300', 'G0600430', 'G3601050', 'G2100730', 'G5300770', 'G4803370', 'G3100090', 'G0500330', 'G4700910', 'G4700290', 'G0600790', 'G0201050', 'G3000530', 'G2100170', 'G5300490', 'G2801090', 'G0601030', 'G4100410', 'G4804110', 'G1200150', 'G2100970', 'G5300590', 'G2102150', 'G3800470', 'G4804070', 'G1600790', 'G1200550', 'G3101390', 'G1300890', 'G4701270', 'G2200810', 'G5300550', 'G1301610', 'G2100510', 'G4200990', 'G5107700', 'G2000610', 'G0201980', 'G4100150', 'G5300390', 'G3000050', 'G3801010', 'G1201290', 'G0500170', 'G0600750', 'G0501170', 'G0500990', 'G4701070', 'G5101010', 'G3700970', 'G1301930', 'G3701510'])

In [62]:
child_df = df3[df3['is_parent'] == 0]

## Find Children for each Parent

In [63]:
parent_child_map = {}
i = 1
for cluster_id, parent in cluster_parent_map.items():
    children = list(child_df[child_df['cluster_id'] == cluster_id]['gis_join'])
    parent_child_map[parent] = children
    print(f'{i}. {parent}: {len(children)}')
    i += 1

1. G5500290: 114
2. G1800350: 85
3. G5300570: 5
4. G3101070: 105
5. G0201300: 1
6. G0600430: 37
7. G3601050: 103
8. G2100730: 32
9. G5300770: 20
10. G4803370: 115
11. G3100090: 70
12. G0500330: 4
13. G4700910: 115
14. G4700290: 39
15. G0600790: 67
16. G0201050: 2
17. G3000530: 17
18. G2100170: 16
19. G5300490: 0
20. G2801090: 120
21. G0601030: 11
22. G4100410: 4
23. G4804110: 117
24. G1200150: 94
25. G2100970: 111
26. G5300590: 12
27. G2102150: 60
28. G3800470: 79
29. G4804070: 125
30. G1600790: 25
31. G1200550: 101
32. G3101390: 80
33. G1300890: 18
34. G4701270: 3
35. G2200810: 121
36. G5300550: 0
37. G1301610: 7
38. G2100510: 26
39. G4200990: 100
40. G5107700: 0
41. G2000610: 0
42. G0201980: 2
43. G4100150: 11
44. G5300390: 18
45. G3000050: 17
46. G3801010: 60
47. G1201290: 26
48. G0500170: 62
49. G0600750: 0
50. G0501170: 70
51. G0500990: 106
52. G4701070: 99
53. G5101010: 113
54. G3700970: 99
55. G1301930: 59
56. G3701510: 91


In [64]:
pickle.dump(parent_child_map, open('pickles/parent_child_map.pkl', 'wb'))

## Find Parent for each Child

In [65]:
children = list(df3[df3['is_parent'] == 0]['gis_join'])

child_parent_map = {}
for child in children:
    cluster_id = df3[df3['gis_join'] == child]['cluster_id'].item()
    parent = cluster_parent_map[cluster_id]
    child_parent_map[child] = parent
    
# child_parent_map.values().unique()

In [66]:
len(child_parent_map.keys())

2994

In [67]:
child_parent_map['G5600150']

'G5500290'

In [68]:
values = child_parent_map.values()
len(set(values))

51

In [69]:
# serialize child_parent_map
pickle.dump(child_parent_map, open('pickles/covid_child_parent_map.pkl', 'wb'))

In [70]:
# Write child parent map to csv
children = []
parents = []
for child, parent in child_parent_map.items():
    children.append(child)
    parents.append(parent)
    
df_child_parent = pd.DataFrame(zip(children, parents), columns=['child', 'parent'])
df_child_parent.to_csv('noaa_child_parent_map.csv', index=False)

In [71]:
# Child Parent Map Sanity Check
unique_children = list(set(children))
unique_parents2 = list(set(parents))

print(len(unique_children))
print(len(unique_parents2))

2994
51


In [72]:
len(set(unique_parents1) - set(unique_parents2))

5

In [73]:
df_final = df3
df_final.to_csv('./clusters-noaa.csv')